In [7]:
import socket
import threading
import json
from cryptography.fernet import Fernet

# Generate an encryption key (store this securely in a real application)
KEY = Fernet.generate_key()
cipher = Fernet(KEY)

# User authentication data (in a real system, use a database)
users = {"user1": "password1", "user2": "password2"}
active_users = {}  # Stores username and IP info

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_address = ('localhost', 65433)
server_socket.bind(server_address)

print("Server is running...")

def decrypt_message(encrypted_message):
    return cipher.decrypt(encrypted_message).decode()

def handle_client():
    while True:
        data, client_address = server_socket.recvfrom(4096)
        try:
            message = json.loads(data.decode())
            if message.get("type") == "auth":
                username = message.get("username")
                password = message.get("password")
                if users.get(username) == password:
                    active_users[username] = client_address
                    response = {"status": "success", "message": "Authenticated"}
                else:
                    response = {"status": "fail", "message": "Invalid credentials"}
                server_socket.sendto(json.dumps(response).encode(), client_address)
            elif message.get("type") == "chat":
                sender = message.get("username")
                encrypted_msg = message.get("message").encode()
                decrypted_msg = decrypt_message(encrypted_msg)
                print(f"{sender} ({client_address}): {decrypted_msg}")
                for user, addr in active_users.items():
                    if addr != client_address:
                        server_socket.sendto(json.dumps({"sender": sender, "message": encrypted_msg.decode()}).encode(), addr)
        except Exception as e:
            print(f"Error: {e}")

threading.Thread(target=handle_client, daemon=True).start()

ModuleNotFoundError: No module named 'cryptography'